In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor as RFC
import warnings 
%matplotlib inline
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
from sklearn import preprocessing
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_excel('Molecular_Descriptor.xlsx',engine='openpyxl')

In [ ]:
data.head(5)

In [ ]:
#删除全0列
data_delzero = data.loc[:,(data!=0).any(axis=0)]
#保存到xlsx中
# data_delzero.to_excel('Molecular_Descriptor_delzero.xlsx')

In [ ]:
data_delzero.drop(['SMILES'],axis=1,inplace=True)

In [ ]:
data_delzero.head(5)

In [ ]:
#归一化
m_scale = preprocessing.MinMaxScaler()
data_delzero_norm = pd.DataFrame(m_scale.fit_transform(data_delzero))

In [ ]:
data_delzero_norm.head(5)

In [ ]:
#写入xlsx文件
# data_delzero_norm.to_excel('Molecular_Descriptor_delzero_norm.xlsx')

In [ ]:
#读入标签
data_label = pd.read_excel('ER_activity.xlsx',engine ='openpyxl')
data_label.drop(columns=['SMILES'],inplace=True)
#归一化
data_label = pd.DataFrame(m_scale.fit_transform(data_label))

In [ ]:
data_label[1]

In [ ]:
#data_label 分布图
# import matplotlib.pyplot as plt
# plt.hist(data_label['pIC50'],orientation='vertical',histtype='bar',
#          color='blue')
# plt.show()

In [ ]:
# plt.hist(data_label['IC50_nM'],orientation='vertical',histtype='bar',
#          color='blue')
# plt.show()

In [ ]:
#数据描述
#未归一化之前
data_scribe = data_delzero.describe(include='all')
data_scribe.to_excel('Molecular_Descriptor_delzero_describe.xlsx')
#归一化之后
data_scribe = data_delzero.describe(include='all')
data_scribe.to_excel('Molecular_Descriptor_delzero_norm_describe.xlsx')

In [ ]:
#随机森林特征排序
rf_model = RFC(n_estimators=1000,max_depth=100,max_features=500)
rf_model.fit(data_delzero_norm,data_label[1])

In [ ]:
features = list(data_delzero.columns.values)
features_importance = rf_model.feature_importances_
features_data_delzero_norm = pd.DataFrame({'Features':features,'Importance':features_importance})
features_data_delzero_norm.sort_values('Importance',inplace=True,ascending=False)

In [ ]:
features_data_delzero_norm.to_excel('Feature_importance_RF.xlsx')

In [ ]:
#相关系数特征排序
#读取归一化后的带有列名的源数据
data_delzero_norm_2 = pd.read_excel('Molecular_Descriptor_delzero_norm_2.xlsx',index_col=0)
data_label.columns = ['IC50_nM','pIC50']
data_final = pd.concat([data_delzero_norm_2,data_label['pIC50']],axis=1)

In [ ]:
data_final.head(5)

In [ ]:
# data_final.drop(columns=['SMILES'],inplace=True)
# data_final.loc[:,~data_final.columns.str.contains("^Unnamed")]
correlation = data_final.corr()
data_corr = correlation['pIC50'].sort_values(ascending=False)

In [ ]:
data_corr

In [ ]:
data_corr.to_excel('Feature_importance_corr.xlsx')

In [ ]:
features_data_delzero_norm

In [ ]:
MDEC-23
MLogP
LipoaffinityIndex
minsOH
nC
minsssN
CrippenLogP
maxHsOH
C1SP2
minHsOH
maxsOH
nT6Ring
n6Ring
BCUTp-1h
C2SP2
hmin
AMR
SwHBa
maxsssN
MDEC-22


In [ ]:
#特征热图
#
data_feature = pd.DataFrame(data_final,columns=['MDEC-23','MLogP','LipoaffinityIndex','minsOH','nC','minsssN','CrippenLogP',
                                               'maxHsOH','C1SP2','minHsOH','maxsOH','nT6Ring','n6Ring','BCUTp-1h',
                                               'C2SP2','hmin','AMR','SwHBa','maxsssN','MDEC-22'])
data_feature.to_excel('Molecular_Descriptor_select_feature.xlsx')
correlation_feature_top20 = data_feature.corr()
correlation_feature_top20

In [ ]:
#特征选取与训练
import seaborn as sns
import matplotlib.pyplot as plt
f,ax=plt.subplots(figsize=(20,20))
plt.title('Top20 Feature Correlation analysis',y=1,size=16)
pl = sns.heatmap(correlation_feature_top20,square=True,vmax=0.8)

In [ ]:
s1 = pl.get_figure()
s1.savefig('HeatMap.png',dpi=300,bbox_inches='tight')

In [ ]:
#KMO检验
import math
def kmo(dataset_corr):
        corr_inv = np.linalg.inv(dataset_corr)
        nrow_inv_corr, ncol_inv_corr = dataset_corr.shape
        A = np.ones((nrow_inv_corr, ncol_inv_corr))
        for i in range(0, nrow_inv_corr, 1):
            for j in range(i, ncol_inv_corr, 1):
                A[i, j] = -(corr_inv[i, j]) / (math.sqrt(corr_inv[i, i] * corr_inv[j, j]))
                A[j, i] = A[i, j]
        dataset_corr = np.asarray(dataset_corr)
        kmo_num = np.sum(np.square(dataset_corr)) - np.sum(np.square(np.diagonal(A)))
        kmo_denom = kmo_num + np.sum(np.square(A)) - np.sum(np.square(np.diagonal(A)))
        kmo_value = kmo_num / kmo_denom
        return kmo_value

print("\nKMO测度:", kmo(correlation_feature_top20))

In [ ]:
# #导入test
# data_test = pd.read_excel('Molecular_Descriptor.xlsx',sheet_name='test',index_col=0)
# data_test.head(5)

In [ ]:
# data_test_feature = pd.DataFrame(data_test,columns=['MDEC-23','MLogP','LipoaffinityIndex','minsOH','nC','minsssN','CrippenLogP',
#                                                'maxHsOH','C1SP2','minHsOH','maxsOH','nT6Ring','n6Ring','BCUTp-1h',
#                                                'C2SP2','hmin','AMR','SwHBa','maxsssN','MDEC-22'])

In [ ]:
#归一化
# data_test_feature_norm = pd.DataFrame(m_scale.fit_transform(data_test_feature))
# data_test_feature_norm.head(5)

In [ ]:
# data_test_feature_norm.to_excel('Molecular_Descriptor_test_feature.xlsx')

In [ ]:
data_test_feature_norm = pd.read_excel('Molecular_Descriptor_test_feature.xlsx')

In [ ]:
data_test_feature_fac = pd.DataFrame()
data_test_feature_fac['fac1'] = data_test_feature_norm['MLogP']*0.895+\
data_test_feature_norm['MDEC-23']*0.891+\
data_test_feature_norm['LipoaffinityIndex']*0.837+\
data_test_feature_norm['nC']*0.836+\
data_test_feature_norm['nT6Ring']*0.835+\
data_test_feature_norm['n6Ring']*0.824+\
data_test_feature_norm['AMR']*0.8+\
data_test_feature_norm['SwHBa']*0.782+\
data_test_feature_norm['hmin']*(-0.738)+\
data_test_feature_norm['C2SP2']*0.735+\
data_test_feature_norm['CrippenLogP']*0.714+\
data_test_feature_norm['MDEC-22']*0.708+\
data_test_feature_norm['minsssN']*0.701+\
data_test_feature_norm['maxsssN']*0.700+\
data_test_feature_norm['BCUTp-1h']*0.604+\
data_test_feature_norm['maxsOH']*0.391+\
data_test_feature_norm['minsOH']*0.393+\
data_test_feature_norm['C1SP2']*(-0.364)

data_test_feature_fac['fac2'] = data_test_feature_norm['AMR']*(-0.346)+\
data_test_feature_norm['maxHsOH']*0.917+\
data_test_feature_norm['minHsOH']*0.916+\
data_test_feature_norm['maxsOH']*0.878+\
data_test_feature_norm['minsOH']*0.877+\
data_test_feature_norm['C1SP2']*(-0.498)

data_test_feature_fac['fac3'] = data_test_feature_norm['CrippenLogP']*(-0.453)+\
data_test_feature_norm['minsssN']*0.621+\
data_test_feature_norm['maxsssN']*0.625

data_test_feature_fac['fac4'] = data_test_feature_norm['nC']*0.388+\
data_test_feature_norm['AMR']*0.434+\
data_test_feature_norm['C1SP2']*0.654

In [ ]:
data_train_feature_norm = data_feature
data_train_feature_fac = pd.DataFrame()
data_train_feature_fac['fac1'] = data_train_feature_norm['MLogP']*0.895+\
data_train_feature_norm['MDEC-23']*0.891+\
data_train_feature_norm['LipoaffinityIndex']*0.837+\
data_train_feature_norm['nC']*0.836+\
data_train_feature_norm['nT6Ring']*0.835+\
data_train_feature_norm['n6Ring']*0.824+\
data_train_feature_norm['AMR']*0.8+\
data_train_feature_norm['SwHBa']*0.782+\
data_train_feature_norm['hmin']*(-0.738)+\
data_train_feature_norm['C2SP2']*0.735+\
data_train_feature_norm['CrippenLogP']*0.714+\
data_train_feature_norm['MDEC-22']*0.708+\
data_train_feature_norm['minsssN']*0.701+\
data_train_feature_norm['maxsssN']*0.700+\
data_train_feature_norm['BCUTp-1h']*0.604+\
data_train_feature_norm['maxsOH']*0.391+\
data_train_feature_norm['minsOH']*0.393+\
data_train_feature_norm['C1SP2']*(-0.364)

data_train_feature_fac['fac2'] = data_train_feature_norm['AMR']*(-0.346)+\
data_train_feature_norm['maxHsOH']*0.917+\
data_train_feature_norm['minHsOH']*0.916+\
data_train_feature_norm['maxsOH']*0.878+\
data_train_feature_norm['minsOH']*0.877+\
data_train_feature_norm['C1SP2']*(-0.498)

data_train_feature_fac['fac3'] = data_train_feature_norm['CrippenLogP']*(-0.453)+\
data_train_feature_norm['minsssN']*0.621+\
data_train_feature_norm['maxsssN']*0.625

data_train_feature_fac['fac4'] = data_train_feature_norm['nC']*0.388+\
data_train_feature_norm['AMR']*0.434+\
data_train_feature_norm['C1SP2']*0.654

In [ ]:
data_labels = pd.read_excel('ER_activity.xlsx',engine ='openpyxl')
data_labels.drop(columns=['SMILES'],inplace=True)
#归一化
data_labels = pd.DataFrame(m_scale.fit_transform(data_labels))

In [ ]:
data_labels.head(5)

In [ ]:
#构建回归模型
# from sklearn import train_test_split
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.neural_network import MLPRegressor
# RMSE评分
def rmsle_cv(model):
    kf = KFold(n_splits=5,shuffle=True,random_state=42).get_n_splits(X_train.values)
    mse = -cross_val_score(model,X_train.values,Y_train,scoring='neg_mean_squared_error',cv=kf)
    return (mse)
from sklearn.metrics import mean_absolute_error
#train val 划分
# data_train_MLP = data_train_feature_fac.head()
data_labels.columns = ['IC50_nM','pIC50']
X_train, X_test, Y_train, Y_test = train_test_split(data_train_feature_fac, data_labels['pIC50'], test_size=0.3, shuffle=True)
# mgr = make_pipeline(RobustScaler(),MLPRegressor(solver='adam', hidden_layer_sizes=(1024,512),activation='relu' ,max_iter=5000))
# score = rmsle_cv(mgr)

In [ ]:
print('\nMLP score:{:.4f}({:.4f})'.format(score.mean(),score.std()))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
rfr = RandomForestRegressor(n_estimators=1000,max_depth=100)
rfr.fit(X_train,Y_train)
y_pre = rfr.predict(X_test)
# rfr.score(X_test,Y_test,scoring='neg_mean_squared_error')
# mse = metrics.mean_squared_error(X_test,Y_test)

In [ ]:
mse = metrics.mean_squared_error(y_pre,Y_test)

In [ ]:
np.sqrt(mse)

In [ ]:
print('\nRFR score:{:.4f}({:.4f})'.format(score_rfr.mean(),score_rfr.std()))

In [ ]:
#构建模型
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(solver='adam', hidden_layer_sizes=(1024,512), max_iter=5000).fit(X_train,Y_train)
regr.score(X_test,Y_test)

In [ ]:
l_svr = SVR(kernel='linear')
l_svr.fit(X_train,Y_train)
l_svr.score(X_test,Y_test)

In [ ]:
l_svr = SVR(kernel='poly')
l_svr.fit(X_train,Y_train)
l_svr.score(X_test,Y_test)

In [ ]:
l_svr = SVR(kernel='rbf')
l_svr.fit(X_train,Y_train)
l_svr.score(X_test,Y_test)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(weights="uniform")
knn.fit(X_train,Y_train)
knn.score(X_test,Y_test)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
dt.fit(X_train,Y_train)
dt.score(X_test,Y_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_train,Y_train)
rfr.score(X_test,Y_test)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
etr = ExtraTreesRegressor()
etr.fit(X_train,Y_train)
etr.score(X_test,Y_test)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor()
gbr.fit(X_train,Y_train)
gbr.score(X_test,Y_test)

In [ ]:
# #导入test
data_trains = pd.read_excel('Molecular_Descriptor.xlsx',sheet_name='training')
data_trains_feature = pd.DataFrame(data_trains,columns=['MDEC-23','MLogP','LipoaffinityIndex','minsOH','nC','minsssN','CrippenLogP',
                                               'maxHsOH','C1SP2','minHsOH','maxsOH','nT6Ring','n6Ring','BCUTp-1h',
                                               'C2SP2','hmin','AMR','SwHBa','maxsssN','MDEC-22'])
data_trains_feature_norm = pd.DataFrame(m_scale.fit_transform(data_trains_feature))
data_trains_feature_norm.head(5)

In [ ]:
regr_final = MLPRegressor(solver='adam', hidden_layer_sizes=(1024,512),activation='relu' ,max_iter=5000).fit(data_trains_feature_norm,data_labels['IC50_nM'])
# regr_final.score(data_train_feature_fac,data_label['pIC50'])

In [ ]:
data_test_feature_norm.head(5)

In [ ]:
# data_test_feature_norm.loc[:,~data_test_feature_norm.columns.str.contains('^Unnamed:0')]
y_pre_rgr_final = regr_final.predict(data_test_feature_norm)

In [ ]:
y_pre_rgr_final = pd.DataFrame(y_pre_rgr_final)
y_pre_rgr_final.columns = ['IC50_nM_test_rgr']
y_pre_rgr_final.to_excel('test_result_rgr_IC50.xlsx')

In [ ]:
rfr_final = RandomForestRegressor()
rfr_final.fit(data_train_feature_fac,data_labels['IC50_nM'])
y_pre_rfr_final = pd.DataFrame(rfr_final.predict(data_test_feature_fac))
y_pre_rfr_final.columns = ['IC50_nM']
y_pre_rfr_final.to_excel('IC50_result_rf.xlsx')

In [ ]:
y_pre = regr.predict(data_test_feature_fac)

In [ ]:
y_pre = pd.DataFrame(y_pre)

In [ ]:
y_pre.columns = ['pIC50_test']

In [ ]:
y_pre.to_excel('test_result.xlsx')

In [ ]:
y_pre_rfr= pd.DataFrame(rfr.predict(data_test_feature_fac))
y_pre_rfr.columns = ['pIC50_test_rf']
y_pre_rfr.to_excel('test_result_rf.xlsx')

In [ ]:
#转换成IC50
